# ArcPy: Accessing Microsoft Planetary Computer (MPC) Data via STAC API

In this notebook, you will:
- Connect to cloud satellite data with ArcPy’s STAC tools
- Explore collections and assets in the MPC STAC Catalog
- Run a custom search for imagery by space, time, and sensor
- Load and inspect results as RasterCollections
- Practice customizing band/processing choices

## 1. Setup

First, import required ArcPy modules. For many cloud datasets, an **Azure Cloud Storage (ACS) connection file** is required (see [Supported STAC Collections](https://pro.arcgis.com/en/pro-app/latest/help/data/imagery/supported-stac-collections.htm)).

In [1]:
import arcpy
from arcpy import AIO

### Set up an ACS connection file for your collection

- Download the right `.acs` file for the dataset (see ArcGIS/Esri help link above).
- Update the path below as needed.

In [2]:
# Required for certain cloud collections ONLY:
acs_path = r'./ACS_FIles/esrims_pc_landsat-c2-l2.acs'  # CHANGE if yours is elsewhere!
a = AIO(acs_path)

## 2. Explore the MPC STAC Catalog

Use `GetSTACInfo` to browse available datasets and collections, and to inspect the details of a specific collection (e.g., Landsat-8).

In [3]:
# Return root info from MPC STAC Catalog
stac_url = 'https://planetarycomputer.microsoft.com/api/stac/v1'
stac_info = arcpy.GetSTACInfo(stac_url, verbose=False)
print('Available Collections:')
print(stac_info['collections'])

Available Collections:
['daymet-annual-pr', 'daymet-daily-hi', '3dep-seamless', '3dep-lidar-dsm', 'fia', 'sentinel-1-rtc', 'gridmet', 'daymet-annual-na', 'daymet-monthly-na', 'daymet-annual-hi', 'daymet-monthly-hi', 'daymet-monthly-pr', 'gnatsgo-tables', 'hgb', 'cop-dem-glo-30', 'cop-dem-glo-90', 'terraclimate', 'nasa-nex-gddp-cmip6', 'gpm-imerg-hhr', 'gnatsgo-rasters', '3dep-lidar-hag', 'io-lulc-annual-v02', 'goes-cmi', 'conus404', '3dep-lidar-intensity', '3dep-lidar-pointsourceid', 'mtbs', 'noaa-c-cap', '3dep-lidar-copc', 'modis-64A1-061', 'alos-fnf-mosaic', '3dep-lidar-returns', 'mobi', 'landsat-c2-l2', 'era5-pds', 'chloris-biomass', 'kaza-hydroforecast', 'planet-nicfi-analytic', 'modis-17A2H-061', 'modis-11A2-061', 'daymet-daily-pr', '3dep-lidar-dtm-native', '3dep-lidar-classification', '3dep-lidar-dtm', 'gap', 'modis-17A2HGF-061', 'planet-nicfi-visual', 'gbif', 'modis-17A3HGF-061', 'modis-09A1-061', 'alos-dem', 'alos-palsar-mosaic', 'deltares-water-availability', 'modis-16A3GF-061

In [12]:
# Get detailed info for Landsat collection 2 level 2
landsat_col_url = stac_url + '/collections/landsat-c2-l2'
collection_info = arcpy.GetSTACInfo(landsat_col_url, verbose=True)
collection_info

{'type': 'Collection',
 'id': 'landsat-c2-l2',
 'title': 'Landsat Collection 2 Level-2',
 'description': 'Landsat Collection 2 Level-2 [Science Products](https://www.usgs.gov/landsat-missions/landsat-collection-2-level-2-science-products), consisting of atmospherically corrected [surface reflectance](https://www.usgs.gov/landsat-missions/landsat-collection-2-surface-reflectance) and [surface temperature](https://www.usgs.gov/landsat-missions/landsat-collection-2-surface-temperature) image data. Collection 2 Level-2 Science Products are available from August 22, 1982 to present.\n\nThis dataset represents the global archive of Level-2 data from [Landsat Collection 2](https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collection-2) acquired by the [Thematic Mapper](https://landsat.gsfc.nasa.gov/thematic-mapper/) onboard Landsat 4 and 5, the [Enhanced Thematic Mapper](https://landsat.gsfc.nasa.gov/the-enhanced-thematic-mapper-plus-etm/) onboard Landsat 7, and the [Operatational

In [5]:
# Show available data "assets" (bands/products) in collection
collection_info = arcpy.GetSTACInfo(landsat_col_url, verbose=False)
print(collection_info['item_assets'])

['qa', 'ang', 'red', 'blue', 'drad', 'emis', 'emsd', 'lwir', 'trad', 'urad', 'atran', 'cdist', 'green', 'nir08', 'lwir11', 'swir16', 'swir22', 'coastal', 'mtl.txt', 'mtl.xml', 'cloud_qa', 'mtl.json', 'qa_pixel', 'qa_radsat', 'qa_aerosol', 'atmos_opacity']


## 3. Define a Query for Search

A typical STAC search includes the following parameters:
- `collections`: the dataset you want
- `bbox`: bounding box in `[xmin, ymin, xmax, ymax]` (WGS84)
- `query`: platform, band, product etc
- `datetime`: time window
- `limit`: max results

[Find more accepted parameters here (OGC STAC docs)](https://docs.ogc.org/DRAFTS/20-004.html#core-query-parameters)

In [6]:
# Example: search for Landsat-8 images over Hawaii in 2022
query = {
    "collections": ["landsat-c2-l2"],
    "bbox": [-156.127, 18.871, -154.776, 20.299],  # Hawaii
    "query": {"platform": {"in": ["landsat-8"]}},
    "datetime": "2022-01-01/2022-12-31",
    "limit": 10
}
query

{'collections': ['landsat-c2-l2'],
 'bbox': [-156.127, 18.871, -154.776, 20.299],
 'query': {'platform': {'in': ['landsat-8']}},
 'datetime': '2022-01-01/2022-12-31',
 'limit': 10}

### (Optional) Define Return Attributes
- Choose which result metadata fields you want to retrieve.

In [7]:
attribute_dict = {
    "Name": "id",
    "Cloud Cover": "eo:cloud_cover",
    "StdTime": "datetime",
    "Platform": "platform",
    "Spatial Reference": "proj:epsg",
    "Extent": "bbox"
}
attribute_dict

{'Name': 'id',
 'Cloud Cover': 'eo:cloud_cover',
 'StdTime': 'datetime',
 'Platform': 'platform',
 'Spatial Reference': 'proj:epsg',
 'Extent': 'bbox'}

## 4. Search and Load Results into a RasterCollection

In [8]:
rc = arcpy.ia.RasterCollection.fromSTACAPI(
    stac_api=stac_url,
    query=query,
    attribute_dict=attribute_dict
)
rc

,Raster,Name,Cloud Cover,StdTime,Platform,Spatial Reference,Extent,Geometry
0,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_063047_20221226_02_T1,9.02,2022-12-26T20:49:04.922397Z,landsat-8,32604,"[-157.35802141, 17.72241449, -155.18280684, 19...",<geoprocessing describe geometry object object...
1,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_063046_20221226_02_T1,11.51,2022-12-26T20:48:41.027120Z,landsat-8,32605,"[-157.07689258, 19.14082455, -154.82766548, 21...",<geoprocessing describe geometry object object...
2,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_062047_20221219_02_T2,40.47,2022-12-19T20:42:55.195542Z,landsat-8,32605,"[-155.83444141, 17.7142945, -153.63556681, 19....",<geoprocessing describe geometry object object...
3,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_062046_20221219_02_T2,47.79,2022-12-19T20:42:31.300266Z,landsat-8,32605,"[-155.5115726, 19.15732452, -153.29564546, 21....",<geoprocessing describe geometry object object...
4,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_063047_20221210_02_T1,12.98,2022-12-10T20:49:12.530876Z,landsat-8,32604,"[-157.37499141, 17.72273449, -155.20002684, 19...",<geoprocessing describe geometry object object...
5,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_063046_20221210_02_T1,6.7,2022-12-10T20:48:48.627128Z,landsat-8,32605,"[-157.09419258, 19.14315455, -154.84481548, 21...",<geoprocessing describe geometry object object...
6,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_062047_20221203_02_T2,81.95,2022-12-03T20:43:03.706452Z,landsat-8,32605,"[-155.86024141, 17.7139345, -153.65820681, 19....",<geoprocessing describe geometry object object...
7,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_062046_20221203_02_T1,43.4,2022-12-03T20:42:39.806940Z,landsat-8,32605,"[-155.5346926, 19.15701452, -153.31847546, 21....",<geoprocessing describe geometry object object...
8,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_063047_20221124_02_T1,14.29,2022-11-24T20:49:13.951064Z,landsat-8,32604,"[-157.36934141, 17.72262449, -155.19424684, 19...",<geoprocessing describe geometry object object...
9,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_063046_20221124_02_T1,48.53,2022-11-24T20:48:50.047316Z,landsat-8,32605,"[-157.08842258, 19.14327455, -154.83910548, 21...",<geoprocessing describe geometry object object...


## 5. Customizing Data: Processing Template & Asset Management

Change pre-processing or limit to specific bands.

In [9]:
# Example 1: Use the Surface Reflectance Processing Template (if available)
rc_template = arcpy.ia.RasterCollection.fromSTACAPI(
    stac_api=stac_url,
    query=query,
    attribute_dict=attribute_dict,
    context={"processingTemplate": "Surface Reflectance"}
)
rc_template

,Raster,Name,Cloud Cover,StdTime,Platform,Spatial Reference,Extent,Geometry
0,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_063047_20221226_02_T1,9.02,2022-12-26T20:49:04.922397Z,landsat-8,32604,"[-157.35802141, 17.72241449, -155.18280684, 19...",<geoprocessing describe geometry object object...
1,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_063046_20221226_02_T1,11.51,2022-12-26T20:48:41.027120Z,landsat-8,32605,"[-157.07689258, 19.14082455, -154.82766548, 21...",<geoprocessing describe geometry object object...
2,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_062047_20221219_02_T2,40.47,2022-12-19T20:42:55.195542Z,landsat-8,32605,"[-155.83444141, 17.7142945, -153.63556681, 19....",<geoprocessing describe geometry object object...
3,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_062046_20221219_02_T2,47.79,2022-12-19T20:42:31.300266Z,landsat-8,32605,"[-155.5115726, 19.15732452, -153.29564546, 21....",<geoprocessing describe geometry object object...
4,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_063047_20221210_02_T1,12.98,2022-12-10T20:49:12.530876Z,landsat-8,32604,"[-157.37499141, 17.72273449, -155.20002684, 19...",<geoprocessing describe geometry object object...
5,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_063046_20221210_02_T1,6.7,2022-12-10T20:48:48.627128Z,landsat-8,32605,"[-157.09419258, 19.14315455, -154.84481548, 21...",<geoprocessing describe geometry object object...
6,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_062047_20221203_02_T2,81.95,2022-12-03T20:43:03.706452Z,landsat-8,32605,"[-155.86024141, 17.7139345, -153.65820681, 19....",<geoprocessing describe geometry object object...
7,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_062046_20221203_02_T1,43.4,2022-12-03T20:42:39.806940Z,landsat-8,32605,"[-155.5346926, 19.15701452, -153.31847546, 21....",<geoprocessing describe geometry object object...
8,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_063047_20221124_02_T1,14.29,2022-11-24T20:49:13.951064Z,landsat-8,32604,"[-157.36934141, 17.72262449, -155.19424684, 19...",<geoprocessing describe geometry object object...
9,https://landsateuwest.blob.core.windows.net/la...,LC08_L2SP_063046_20221124_02_T1,48.53,2022-11-24T20:48:50.047316Z,landsat-8,32605,"[-157.08842258, 19.14327455, -154.83910548, 21...",<geoprocessing describe geometry object object...


In [10]:
# Example 2: Get only RGB assets for each scene
rc_asset = arcpy.ia.RasterCollection.fromSTACAPI(
    stac_api=stac_url,
    query=query,
    attribute_dict=attribute_dict,
    context={"assetManagement": ["red", "green", "blue"]}
)
rc_asset

,Raster,Name,Cloud Cover,StdTime,Platform,Spatial Reference,Extent,Geometry
0,C:\Users\col12422\AppData\Local\Temp\22\x2e92b...,LC08_L2SP_063047_20221226_02_T1,9.02,2022-12-26T20:49:04.922397Z,landsat-8,32604,"[-157.35802141, 17.72241449, -155.18280684, 19...",<geoprocessing describe geometry object object...
1,C:\Users\col12422\AppData\Local\Temp\22\x0ebde...,LC08_L2SP_063046_20221226_02_T1,11.51,2022-12-26T20:48:41.027120Z,landsat-8,32605,"[-157.07689258, 19.14082455, -154.82766548, 21...",<geoprocessing describe geometry object object...
2,C:\Users\col12422\AppData\Local\Temp\22\xd03ab...,LC08_L2SP_062047_20221219_02_T2,40.47,2022-12-19T20:42:55.195542Z,landsat-8,32605,"[-155.83444141, 17.7142945, -153.63556681, 19....",<geoprocessing describe geometry object object...
3,C:\Users\col12422\AppData\Local\Temp\22\xd5cfd...,LC08_L2SP_062046_20221219_02_T2,47.79,2022-12-19T20:42:31.300266Z,landsat-8,32605,"[-155.5115726, 19.15732452, -153.29564546, 21....",<geoprocessing describe geometry object object...
4,C:\Users\col12422\AppData\Local\Temp\22\xebcb6...,LC08_L2SP_063047_20221210_02_T1,12.98,2022-12-10T20:49:12.530876Z,landsat-8,32604,"[-157.37499141, 17.72273449, -155.20002684, 19...",<geoprocessing describe geometry object object...
5,C:\Users\col12422\AppData\Local\Temp\22\xd7fb4...,LC08_L2SP_063046_20221210_02_T1,6.7,2022-12-10T20:48:48.627128Z,landsat-8,32605,"[-157.09419258, 19.14315455, -154.84481548, 21...",<geoprocessing describe geometry object object...
6,C:\Users\col12422\AppData\Local\Temp\22\x93590...,LC08_L2SP_062047_20221203_02_T2,81.95,2022-12-03T20:43:03.706452Z,landsat-8,32605,"[-155.86024141, 17.7139345, -153.65820681, 19....",<geoprocessing describe geometry object object...
7,C:\Users\col12422\AppData\Local\Temp\22\x32051...,LC08_L2SP_062046_20221203_02_T1,43.4,2022-12-03T20:42:39.806940Z,landsat-8,32605,"[-155.5346926, 19.15701452, -153.31847546, 21....",<geoprocessing describe geometry object object...
8,C:\Users\col12422\AppData\Local\Temp\22\x491f9...,LC08_L2SP_063047_20221124_02_T1,14.29,2022-11-24T20:49:13.951064Z,landsat-8,32604,"[-157.36934141, 17.72262449, -155.19424684, 19...",<geoprocessing describe geometry object object...
9,C:\Users\col12422\AppData\Local\Temp\22\x8305f...,LC08_L2SP_063046_20221124_02_T1,48.53,2022-11-24T20:48:50.047316Z,landsat-8,32605,"[-157.08842258, 19.14327455, -154.83910548, 21...",<geoprocessing describe geometry object object...


## 6. Compare Band Names Returned from Each Method

See how different context options affect available raster bands.

In [11]:
print(f"{'Default template (Multiband):':<30} {rc[0]['Raster'].bandNames}")
print(f"{'Surface Reflectance template:':<30} {rc_template[0]['Raster'].bandNames}")
print(f"{'Specific assets:' :<30} {rc_asset[0]['Raster'].bandNames}")

Default template (Multiband):  ['sr_band1', 'sr_band2', 'sr_band3', 'sr_band4', 'sr_band5', 'sr_band6', 'sr_band7', 'st_band10', 'QA']
Surface Reflectance template:  ['sr_band1', 'sr_band2', 'sr_band3', 'sr_band4', 'sr_band5', 'sr_band6', 'sr_band7']
Specific assets:               ['Band_1', 'Band_2', 'Band_3']


## 7. Further Exploration, Discussion, and Challenge

- **How else could you use this workflow?**
    - Try a different collection (e.g., Sentinel-2, NAIP) or location
    - Experiment with the datetime, query, and limit values in your STAC search
- **What could you do with the RasterCollection objects?**
    - Map display in ArcGIS Pro
    - Raster analysis (NDVI, change over time)
    - Tiled raster export

### Challenge
- Modify bbox to your area of interest
- Retrieve imagery for several years and compare results
- Use attribute_dict to retrieve additional metadata (e.g., sun angle, processing level)
- Try with another collection ID (see: `stac_info['collections']`)

## Resources / References

- [Supported STAC Collections & ACS Requirements](https://pro.arcgis.com/en/pro-app/latest/help/data/imagery/supported-stac-collections.htm)
- [Planetary Computer STAC Root](https://planetarycomputer.microsoft.com/api/stac/v1)
- [STAC API Queryables Example (Landsat)](https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2/queryables)
- [ArcGIS Pro Help: Connect to Cloud Raster Data](https://pro.arcgis.com/en/pro-app/latest/help/data/imagery/connect-to-cloud-raster-data.htm)
- [ArcPy Documentation](https://pro.arcgis.com/en/pro-app/latest/arcpy/main/arcgis-pro-arcpy-reference.htm)
- [STAC Specification](https://stacspec.org/)